Attacks

In [1]:
!pip install opacus
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
import tarfile
import torch
import requests
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torchvision
import torch.utils.data as torch_data
import torchvision.transforms as transforms
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time
from google.colab import files

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 3.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
## download the data
response = requests.get(f'https://www.comp.nus.edu.sg/~reza/files/dataset_texas.tgz')
if response.status_code == 200:
    with open(f'dataset_texas.tgz', 'wb') as file:
        file.write(response.content)
    print("Download completed successfully.")
else:
    print(f"Failed to download file: {response.status_code}")

with tarfile.open(f'dataset_texas.tgz') as f:
    f.extractall(f'data/')

with open('data/texas/100/feats', 'r') as f:
    features = f.readlines()
with open('data/texas/100/labels', 'r') as f:
    labels = f.readlines()


Download completed successfully.


In [5]:
## build the sampled trainig and test datasets
features_list = [list(map(int, ''.join(feature.split()).split(','))) for feature in features]
labels_list = [int(label.strip()) for label in labels]
size = int(0.8 * len(features))
sample_size = 500
feat_tens_train = torch.tensor(features_list[:sample_size], dtype=torch.float)
l_tens_train = torch.tensor(labels_list[:sample_size], dtype=torch.long)
feat_tens_test = torch.tensor(features_list[size:sample_size], dtype=torch.float)
l_tens_test = torch.tensor(labels_list[size:sample_size], dtype=torch.long)
dataset_train = TensorDataset(feat_tens_train, l_tens_train)
dataset_test = TensorDataset(feat_tens_test, l_tens_test)

In [7]:
import os
print(os.getcwd())

/content


In [6]:
## load the model
model = torch.load("model_Y.pth")


FileNotFoundError: [Errno 2] No such file or directory: 'model_Y.pth'

In [ ]:
model = 'blabla' ## salvati come pth


x, y = 'sample of n elements'  ## check how to sample from tensor datasets
losses = 'array of losses for model(x), y'
sorted_elem = 'sort losses increasingly'

## assume that 50% of x are in the training set
## pick first n/2 x wrt to sorted loss
## check how many of these are actually in the training dataset

## in case of unknown percentage
max_loss = 'max loss'
threshold = 'between 0 and max_loss, intervall max_loss/100'

## ROc curve, for each threshold comupte how many x are chosen as members
## compute ratio TPR/FPR and plot the curve
